In [1]:
#Importing Necessary  Modules
import re           
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from  sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from  sklearn.feature_extraction.text import TfidfVectorizer,ENGLISH_STOP_WORDS

In [2]:
#Reading Dataset
doc=pd.read_csv('data.csv')
print(doc.info)


<bound method DataFrame.info of                                                  Date           Author  \
0                                        May 12, 2019            Renee   
1                                       June 17, 2019        Tech nerd   
2                                        June 5, 2019         Brittney   
3                                       June 26, 2019            wkags   
4                                        May 16, 2019  joseph thompson   
..                                                ...              ...   
95                                    August 27, 2019              Ace   
96                                       July 3, 2019  Adeyanju Samuel   
97                                   November 4, 2019         Noah C.    
98                                  November 11, 2019     Cody Knappen   
99  Platinum Plan or higher is required in order t...              NaN   

   Verified Helpful                                              Title  \
0    

In [3]:
review=doc[['Rating','Body']]

In [4]:
#droping missing values
review.dropna(inplace=True)
print(review.shape)
review.astype('str')#coverting object type to string type

(99, 2)


C:\Users\SURENDAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Rating,Body
0,5.0,I've only had the a20 for a week but I absolut...
1,5.0,This is pretty much a budget device. That bei...
2,5.0,I really like this phone the graphics is gorge...
3,5.0,This is the 2nd phone I've purchased in the sa...
4,1.0,The Wi-Fi don't work
...,...,...
94,5.0,I love my new phone! Had no issues hooking it ...
95,5.0,"Amoled screen, great battery, Samsung Dex, gre..."
96,5.0,I am not disappointed buying this phone from N...
97,4.0,Purchased for my mother. Great budget phone th...


In [5]:
#Labeling target variable to 0 and 1
def Rating(text):
    if text >= 3.5:
        return 1
    else:
        return 0
review['Rating']=review['Rating'].apply(Rating)
print(review.Rating)

0     1
1     1
2     1
3     1
4     0
     ..
94    1
95    1
96    1
97    1
98    1
Name: Rating, Length: 99, dtype: int64


C:\Users\SURENDAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
#Extracting words  from the 'Body' column
def noise_removal(text):
    return re.sub('[^A-Za-z]+',' ',text)
review['Body']=review['Body'].apply(noise_removal)

C:\Users\SURENDAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#converting text to lowercase
def lower_case(text):
    return text.lower()
review['Body']=review['Body'].apply(lower_case)
print(review)

    Rating                                               Body
0        1  i ve only had the a for a week but i absolutel...
1        1  this is pretty much a budget device that being...
2        1  i really like this phone the graphics is gorge...
3        1  this is the nd phone i ve purchased in the sam...
4        0                               the wi fi don t work
..     ...                                                ...
94       1  i love my new phone had no issues hooking it u...
95       1  amoled screen great battery samsung dex great ...
96       1  i am not disappointed buying this phone from n...
97       1  purchased for my mother great budget phone tha...
98       1  i just got this phone and it s beautiful it ha...

[99 rows x 2 columns]


C:\Users\SURENDAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#tokenizing corpus
def tokenizing(tokens):
    return word_tokenize(tokens)
review['Body']=review['Body'].apply(tokenizing)
print(review)

    Rating                                               Body
0        1  [i, ve, only, had, the, a, for, a, week, but, ...
1        1  [this, is, pretty, much, a, budget, device, th...
2        1  [i, really, like, this, phone, the, graphics, ...
3        1  [this, is, the, nd, phone, i, ve, purchased, i...
4        0                        [the, wi, fi, don, t, work]
..     ...                                                ...
94       1  [i, love, my, new, phone, had, no, issues, hoo...
95       1  [amoled, screen, great, battery, samsung, dex,...
96       1  [i, am, not, disappointed, buying, this, phone...
97       1  [purchased, for, my, mother, great, budget, ph...
98       1  [i, just, got, this, phone, and, it, s, beauti...

[99 rows x 2 columns]


C:\Users\SURENDAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
#Converting tokens to strings
def convert_to_string(text):
    return ' '.join(text)
review['Body']=review['Body'].apply(convert_to_string)
print(review.Body)

0     i ve only had the a for a week but i absolutel...
1     this is pretty much a budget device that being...
2     i really like this phone the graphics is gorge...
3     this is the nd phone i ve purchased in the sam...
4                                  the wi fi don t work
                            ...                        
94    i love my new phone had no issues hooking it u...
95    amoled screen great battery samsung dex great ...
96    i am not disappointed buying this phone from n...
97    purchased for my mother great budget phone tha...
98    i just got this phone and it s beautiful it ha...
Name: Body, Length: 99, dtype: object


C:\Users\SURENDAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#creating Tfidf model to vectorize words in the corpus
vect=TfidfVectorizer(ngram_range=(1,2),stop_words=ENGLISH_STOP_WORDS)
tfidf_model=vect.fit_transform(review.Body)
#converting sparse matrix  to array to create dataframe
x=pd.DataFrame(tfidf_model.toarray(),columns=vect.get_feature_names())
print(x)   
y=review.Rating
print(y)


    aaaall  aaaall little      able  able activate  able better  able connect  \
0      0.0            0.0  0.087894            0.0          0.0           0.0   
1      0.0            0.0  0.000000            0.0          0.0           0.0   
2      0.0            0.0  0.000000            0.0          0.0           0.0   
3      0.0            0.0  0.000000            0.0          0.0           0.0   
4      0.0            0.0  0.000000            0.0          0.0           0.0   
..     ...            ...       ...            ...          ...           ...   
94     0.0            0.0  0.000000            0.0          0.0           0.0   
95     0.0            0.0  0.000000            0.0          0.0           0.0   
96     0.0            0.0  0.000000            0.0          0.0           0.0   
97     0.0            0.0  0.000000            0.0          0.0           0.0   
98     0.0            0.0  0.000000            0.0          0.0           0.0   

    able primarily  able sw

In [11]:
#using logistic regression to fit ML model
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=123,stratify=y)
log_reg=LogisticRegression().fit(x_train,y_train)
y_predict=log_reg.predict(x_test)
print('accuracy score:',accuracy_score(y_predict,y_test))

C:\Users\SURENDAR\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy score: 0.75
